In [49]:
import yaml
confi = '../DCGAN_UNIT_baseline/exps/gray2colorFundus.yaml'
stream = open(confi,'r')
docs = yaml.load_all(stream)
class NetConfig(object):
    def __init__(self, config):
        stream = open(config,'r')
        docs = yaml.load_all(stream)
        for doc in docs:
            for k, v in doc.items():
                if k == "train":
                    for k1, v1 in v.items():
                        cmd = "self." + k1 + "=" + repr(v1)
                        print(cmd)
                        exec(cmd)
        stream.close()

config = NetConfig(confi)  
print(config)

self.snapshot_save_iterations=100000
self.image_save_iterations=50000
self.image_display_iterations=2
self.display=1
self.snapshot_prefix='/home/outputs/'
self.hyperparameters={'trainer': 'COCOGANTrainer', 'lr': 0.0001, 'll_direct_link_w': 100, 'kl_direct_link_w': 0.1, 'll_cycle_link_w': 100, 'kl_cycle_link_w': 0.1, 'gan_w': 10, 'batch_size': 1, 'max_iterations': 4, 'gen': {'name': 'COCOResGen2', 'ch': 64, 'input_dim_a': 3, 'input_dim_b': 3, 'n_enc_front_blk': 3, 'n_enc_res_blk': 3, 'n_enc_shared_blk': 1, 'n_gen_shared_blk': 1, 'n_gen_res_blk': 3, 'n_gen_front_blk': 3}, 'dis': {'name': 'COCOSharedDis', 'ch': 64, 'input_dim_a': 3, 'input_dim_b': 3, 'n_front_layer': 2, 'n_shared_layer': 4}, 'dcgan': {'root_data': './images/data/*', 'image_height': 248, 'image_width': 248, 'num_iterations': 4, 'batch_size': 8, 'save_interval': 2, 'continueTra': 'False', 'train_stepsIni': 0, 'imageType': 'gray', 'root_weights': '../DCGAN_UNIT_baseline/', 'modelSteps': 3}}
self.datasets={'train_a': {'class_

In [52]:
print(config.datasets['train_b']['channels'])

3


In [43]:
import numpy as np
import cv2 as cv

from keras.models import Sequential
from keras.layers import Dense, Activation, Reshape
from keras.layers import Conv2DTranspose, UpSampling2D
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.optimizers import Adam, RMSprop, SGD
from keras import regularizers


class dcgan_gen:
    # __init__ could be considered as the constructor method of the class
    def __init__(self, config): # self is the instance. config and numEx are parameters
        # Hyperparameters
        self.image_width = config.hyperparameters['dcgan']['image_width']
        self.image_height = config.hyperparameters['dcgan']['image_height']
        self.ModelSteps = config.hyperparameters['dcgan']['modelSteps'] 
        self.root_weights = config.hyperparameters['dcgan']['root_weights']

        self.dim = int(self.image_width / 4)
        self.latentVar = 100       # latent variables. DO NOT CHANGE IT
        self.lrL2 = 1e-7  # This value controls the regularizer
        self.dropout = 0.4
        
    def generator(self):        

        G = Sequential()        
        depth = 64+64+64+64
        # In: 1000
        # Out: dim x dim x depth
        G.add(Dense(self.dim*self.dim*depth, input_dim=self.latentVar, kernel_regularizer=regularizers.l2(self.lrL2)))
        G.add(BatchNormalization(momentum=0.9))
        G.add(Activation('relu'))
        G.add(Reshape((self.dim, self.dim, depth)))
        G.add(Dropout(self.dropout))

        # In: dim x dim x depth
        # Out: 2*dim x 2*dim x depth/2
        G.add(UpSampling2D(size=(2, 2)))
        G.add(Conv2DTranspose(int(depth/2), 5, padding='same', kernel_regularizer=regularizers.l2(self.lrL2)))
        G.add(BatchNormalization(momentum=0.9))
        G.add(Activation('relu'))

        G.add(UpSampling2D(size=(2, 2)))
        G.add(Conv2DTranspose(int(depth/4), 5, padding='same', kernel_regularizer=regularizers.l2(self.lrL2)))
        G.add(BatchNormalization(momentum=0.9))
        G.add(Activation('relu'))

        G.add(Conv2DTranspose(int(depth/8), 5, padding='same', kernel_regularizer=regularizers.l2(self.lrL2)))
        G.add(BatchNormalization(momentum=0.9))
        G.add(Activation('relu'))

        # Out: image_width x image_height x 1 grayscale image [0.0,1.0] per pix
        G.add(Conv2DTranspose(1, 5, padding='same', kernel_regularizer=regularizers.l2(self.lrL2)))
        G.add(Activation('tanh'))

        return G
        
    def fake_image(self, noise_input): # Each method within a class automatically takes the instance as the first argument
                           # The instance and numEx are the only parameters we need in this case
                        # numEx is the number of noise samples

        G = self.generator()
        # We first check if the weights exist
        weights_fileG = self.root_weights + 'generator_' + str(self.ModelSteps) + '_steps.h5'
        G.load_weights(weights_fileG)

        # predict using the noise_input matrix. noise_input matrix contains all the generated samples
        image_fake = G.predict(noise_input)       
        
        return image_fake
    

In [44]:
tg = dcgan_gen(config)

In [45]:
# Here we generate the random samples
noise_input = np.random.normal(size=[90000, 100])
print(noise_input.shape)
f = tg.fake_image(np.array([noise_input[0,:]]))
print(f[0,:,:,:].shape)

(90000, 100)
(248, 248, 1)


In [56]:
# Datagenerator using tensors. I HAVE NO IDEA WHY WE NEED TWO ARRAYS. TARGETS ARE THE LABELS?
import torch.utils.data as utils
import torch

my_x = [np.array([1,2]),np.array([5,6])] # a list of numpy arrays
print(len(my_x))
my_y = [np.array([3.]), np.array([2.])] # another list of numpy arrays (targets)
print(len(my_y))
tensor_x = torch.stack([torch.Tensor(i) for i in my_x]) # transform to torch tensors
tensor_y = torch.stack([torch.Tensor(i) for i in my_y])

my_dataset = utils.TensorDataset(tensor_x,tensor_y) # create your datset
my_dataloader = utils.DataLoader(my_dataset) # create your dataloader

2
2


In [57]:
for i in my_dataloader:
    print(i)

[
 1  2
[torch.FloatTensor of size 1x2]
, 
 3
[torch.FloatTensor of size 1x1]
]
[
 5  6
[torch.FloatTensor of size 1x2]
, 
 2
[torch.FloatTensor of size 1x1]
]
